<a href="https://colab.research.google.com/github/Tejaswidarsi/SHA-512/blob/main/Lab_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1) Write a program to generate Hash code for a given text using SHA 512

In [ ]:
import hashlib

def generate_sha512_hash(text):
    sha512_hash = hashlib.sha512(text.encode()).hexdigest()
    return sha512_hash

text = input("Enter the text: ")
print("SHA-512 Hash:", generate_sha512_hash(text))


Enter the text: Hello
SHA-512 Hash: 3615f80c9d293ed7402687f94b22d58e529b8cc7916f8fac7fddf7fbd5af4cf777d3d795a7a00a16bf7e7f3fb9561ee9baae480da9fe7a18769e71886b03f315


2) Write a program to send a message ensuring confidentiality and integrity
Hint for 2:
i) perform diffie hellman exchange (from previous lab) to generate a session key<br>
ii) Compute Hash code of the message text<br>
iii) use AES to encrypt the (text + hash code)<br>
iv) encrypt the (message + Hash Code)<br>
v) send it to client/server<br>
vi) decrypt the message<br>
vii) check the integrity.<br>

In [ ]:
import hashlib
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend

private_A = ec.generate_private_key(ec.SECP256R1())
private_B = ec.generate_private_key(ec.SECP256R1())

public_A = private_A.public_key()
public_B = private_B.public_key()


shared_secret_A = private_A.exchange(ec.ECDH(), public_B)
shared_secret_B = private_B.exchange(ec.ECDH(), public_A)

assert shared_secret_A == shared_secret_B, "Key exchange failed!"
aes_key = shared_secret_A[:16]


def compute_hash(msg):
    return hashlib.sha512(msg.encode()).hexdigest()

def encrypt_message(msg, key):
    msg_hash = compute_hash(msg)
    combined_data = (msg + msg_hash).encode()


    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    padded_data = padder.update(combined_data) + padder.finalize()

    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
    encryptor = cipher.encryptor()

    ciphertext = encryptor.update(padded_data) + encryptor.finalize()
    return ciphertext

def decrypt_message(ciphertext, key):
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
    decryptor = cipher.decryptor()

    decrypted_data = decryptor.update(ciphertext) + decryptor.finalize()

    unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
    unpadded_data = unpadder.update(decrypted_data) + unpadder.finalize()

    message = unpadded_data[:-128].decode().strip()
    received_hash = unpadded_data[-128:].decode().strip()

    return message, "Integrity Verified!" if compute_hash(message) == received_hash else "Integrity Check Failed!"


message = "Cryptography!"
ciphertext = encrypt_message(message, aes_key)

print("\nCiphertext (Confidentiality Ensured):", ciphertext.hex())


decrypted_msg, status = decrypt_message(ciphertext, aes_key)

print("\nDecrypted Message:", decrypted_msg)
print("Integrity Status:", status)



Ciphertext (Confidentiality Ensured): 44e0e113a164febb2dce785328469bc16f6f4899411deb81a8ef4f716c5d5e44139e950741cbd2a8e9d331c6adfeea24260e3bf0e1ebbaf717216bc3522468a1716aa72e27abc31e9cbafa972d516f89af27e29f4d733a848f96e6bb47dd5b00ceb0c76728fa2bcc8030ac8dda0634e346961a1579a26cd797a09c441609bc32a6b8cc9e70349615d1fa50128f69b1c0

Decrypted Message: Cryptography!
Integrity Status: Integrity Verified!
